In [276]:
import numpy as np
import open3d as o3d
import os
import zipfile

from sklearn.neighbors import NearestNeighbors

In [8]:
filelist = os.listdir("safaty_doors/tof_photo/")

In [165]:
print("Load a ply point cloud, print it, and render it")
for i in range(20):
    pcd = o3d.io.read_point_cloud("safaty_doors/tof_photo/" + filelist[i])
    #print(pcd)
    #print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd])

Load a ply point cloud, print it, and render it


In [220]:
#pcd = o3d.io.read_point_cloud("safaty_doors/tof_photo/" + filelist[0])
pcd = o3d.io.read_point_cloud("point_cloud_gt/clouds_tof/cloud_0_0065.pcd")
data = np.asarray(pcd.points)
print(data.shape)
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pcd.points)
o3d.visualization.draw_geometries([pcd])

(228588, 3)


In [215]:
def least_squares(points):
   """
   Функция для аппроксимации массива точек прямой, основанная на
   методе наименьших квадратов.

   :param points: Входной массив точек формы [N, 2]
   :return: Numpy массив формы [N, 2] точек на прямой
   """

   x = points[:, 0]
   y = points[:, 1]
   # Для метода наименьших квадратов нам нужно, чтобы X был матрицей,
   # в которой первый столбец - единицы, а второй - x координаты точек
   X = np.vstack((np.ones(x.shape[0]), x)).T
   normal_matrix = np.dot(X.T, X)
   moment_matrix = np.dot(X.T, y)
   # beta_hat это вектор [перехват, наклон], рассчитываем его в
   # в соответствии с формулой.
   beta_hat = np.dot(np.linalg.inv(normal_matrix), moment_matrix)
   intercept = beta_hat[0]
   slope = beta_hat[1]
   # Теперь, когда мы знаем параметры прямой, мы можем
   # легко вычислить y координаты точек на прямой.
   y_hat = intercept + slope * x
   # Соберем x и y в единую матрицу, которую мы собираемся вернуть
   # в качестве результата.
   points_hat = np.vstack((x, y_hat)).T

   return points_hat

In [213]:
new_data = data[data[:,2]>5.8]
new_data[:,:2].shape
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(new_data)
o3d.visualization.draw_geometries([pcd1])

In [197]:
new_data

array([[ 0.147     , -0.30599999,  5.9150004 ],
       [ 0.69599998, -0.30000001,  5.8222504 ],
       [-0.43799999, -0.29699999,  5.8930001 ],
       [-0.22500001, -0.264     ,  6.6995001 ],
       [-0.222     , -0.264     ,  6.5702505 ],
       [-0.58499998, -0.255     ,  6.2972503 ],
       [ 0.94200003, -0.252     ,  6.0052505 ],
       [ 0.093     , -0.23100001,  7.0100002 ]])

In [212]:
y = least_squares(new_data[:,:2])

LinAlgError: Singular matrix

In [199]:
print("Уравнение прямой, проходящей через эти точки:")
k = (y[0][1] - y[-1][1]) / (y[0][0] - y[-1][0])
b = y[-1][1] - k*y[-1][0]
print(" y = %.2f*x + %.2f" % (k, b))

Уравнение прямой, проходящей через эти точки:
 y = -0.00*x + -0.27


In [335]:
lstfile = os.listdir("point_cloud_gt/clouds_tof")
for i,f in enumerate(lstfile):
    pcd = o3d.io.read_point_cloud("point_cloud_gt/clouds_tof/"+f)
    pcd_colors = np.asarray(pcd.colors)
    pcd_points = np.asarray(pcd.points)
    neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
    neigh.fit(pcd_points)
    distances, indices = neigh.kneighbors(pcd_points)
    distances = distances[:,1].reshape((pcd_points.shape[0]))
    pcd_points = pcd_points[distances < 0.01]
    pcd.points = o3d.utility.Vector3dVector(pcd_points)
    
    
    mesh_box1 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                    height=.005,
                                                    depth=4).translate((-1, -0.22, 2))

    R = mesh_box1.get_rotation_matrix_from_xyz((0.0, 0.0,0.01))
    mesh_box1.rotate(R, center=(-1, -0.22, 3))

    mesh_box2 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                    height=.005,
                                                    depth=1.5).translate((-1, -0.36, 4.5))

    R = mesh_box2.get_rotation_matrix_from_xyz((-0.07, 0, 0))
    mesh_box2.rotate(R, center=(-1, -0.36, 3)).translate((0, -0.08, 0))

    
    mesh_box3 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                    height=.005,
                                                    depth=2.5).translate((-1, -0.36, 1.5))

    R = mesh_box3.get_rotation_matrix_from_xyz((-0.15, 0, 0))
    mesh_box3.rotate(R, center=(-1, -0.36, 3)).translate((0, -0.125, 0.51))
    

    mesh_box1.compute_vertex_normals()
    mesh_box1.paint_uniform_color([0.4, 0.1, 0.1])

    mesh_box2.compute_vertex_normals()
    mesh_box2.paint_uniform_color([0.4, 0.1, 0.1])

    mesh_box3.compute_vertex_normals()
    mesh_box3.paint_uniform_color([0.4, 0.1, 0.1])
    #pcd1 = o3d.geometry.PointCloud()
    #pcd1.points = o3d.utility.Vector3dVector(data)
    o3d.visualization.draw_geometries([pcd, mesh_box1, mesh_box2,mesh_box3])
    #o3d.visualization.draw_geometries([mesh_box])

KeyboardInterrupt: 

In [269]:
pcd = o3d.io.read_point_cloud("point_cloud_gt/clouds_tof/cloud_0_0003.pcd")
data = np.asarray(pcd.points)
print(data.shape)
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pcd.points)
o3d.visualization.draw_geometries([pcd])

(193877, 3)
